In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)

In [ ]:
from os.path import exists

<a id="anchor_metafields"></a>
# Meta-fields

<a id="anchor_metafields_dataseta"></a>
## [DatasetA](#anchor_dataseta) fields

In [ ]:
### The path to the full dataset with all relevant fields
DATASETA_PATH = "data-compas/compas-scores-two-years.csv"

### The decision source to explain, needs to be 0/1 split
DATASETA_DECISION = "2YCOMPAS" # Original dataset "low-risk/high-risk" threshold split
# DATASETA_DECISION = "RACIST" # Explicitly judging by race group only, [TODO] with the same accuracy vs. same low/high risk split as original?

### Name of column to use in justification
DATASETA_DECISION_COLNAME = "ncol_decision"

### Seed to use in 'ref'(reference,train)/'evl'(evaluate,test) split sampling
DATASETA_SPLIT_SEED = 1
### Ratio of 'train'/'test' split sampling
DATASETA_SPLIT_RATIO = 0.8

<a id="anchor_metafields_datasetb"></a>
## [DatasetB](#anchor_datasetb) fields

In [ ]:
# ### Maximum number of factors to include in the whole-dataset evaluation
# MAX_RATEXPL_FACTORS = 2

<a id="anchor_metafields_explain"></a>
## [Explain](#anchor_explain) fields

<a id="anchor_metafields_evaluate"></a>
## [Evaluate](#anchor_evaluate) fields

In [ ]:
# ### Fields accessible to use in rationalization?
# ### ... some of which are VERY unfair and/or illegal
# ACCESSIBLE_FIELDS = [
#     # 'sex', # >:(
#     # 'race', # >:(
#     # 'age', # >:(
#     # 'age_cat',
#     'juv_fel_count',
#     'juv_misd_count',
#     'juv_other_count',
#     'priors_count',
#     'c_charge_degree',
#     'c_charge_desc',
# ]

# ### Field that is the source of what we are rationalizing based on
# TRUERESULT_FIELD = 'two_year_recid'
# # TRUERESULT_FIELD = 'is_recid'
# ### Field we are trying to rationalize why it could be 1 vs 0
# # JUSTIFYING_FIELD = 'is_recid' # not going to use 'is_violent_recid' much here
# JUSTIFYING_FIELD = 'two_year_recid'
# ### Field that is 1 if justifying==trueresult, otherwise 0.
# ### This is DEFINED BY ME AND NOT IN THE ORIGINAL DATA
# WASCORRECT_FIELD = 'pred_accurate'

# ### Threshold for confidence range
# CONF_ALPHA = 0.05

<a id="anchor_dataseta"></a>
# DatasetA: what decisions were made?

[Relevant metafields](#anchor_metafields_dataseta)

In [ ]:
# see: https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb
def dataseta_compas_filter(df):
    df = df[df["days_b_screening_arrest"] >= -30]
    df = df[df["days_b_screening_arrest"] <= 30]
    df = df[df["is_recid"] != -1]
    df = df[df["c_charge_degree"] != "O"]
    df = df[df["score_text"] != "N/A"]
    return df

In [ ]:
def dataseta_decision_compas(df):
    return df["is_recid"]

In [ ]:
def dataseta_decision_racist(df):
    # Match for the number of positive predictions
    num_pos_pred = sum(df["is_recid"].values)
    # sort by race, apply positives from 1->0 in alphabetic race order
    df = df.sort_values("race")
    df["temp"] = [
        (1 if i<num_pos_pred else 0) 
        for i in range(len(df))
    ]
    # re-order to match original dataset index
    df = df.sort_index()
    return df["temp"]

In [ ]:
# Import dataset
rawsrc_df = pd.read_csv(DATASETA_PATH)

# Filter / preprocess the dataset to remove edge cases
decisions_df = dataseta_compas_filter(rawsrc_df)

# Add model decisions as a custom named column
decisions_ref = {
    "2YCOMPAS": dataseta_decision_compas,
    "RACIST": dataseta_decision_racist,
}
decisions_df[DATASETA_DECISION_COLNAME] = decisions_ref[DATASETA_DECISION](decisions_df)

# [TODO] backup

# Do ref/evl (train/test) split
decisions_ref_df = decisions_df.sample(
    n=int(DATASETA_SPLIT_RATIO*len(decisions_df)),
    random_state=DATASETA_SPLIT_SEED,
)
decisions_evl_df = decisions_df[
    ~decisions_df.index.isin(decisions_ref_df.index)
]

# Print preview of the dataset
decisions_df.shape
decisions_df[:5]
decisions_ref_df.shape
decisions_evl_df.shape

<a id="anchor_datasetb"></a>
# DatasetB: what explanations could be used?

[Relevant metafields](#anchor_metafields_datasetb)

<a id="anchor_explain"></a>
# Explain: what explanations were actually used for each case?

[Relevant metafields](#anchor_metafields_explain)

<a id="anchor_evaluate"></a>
# Evaluate: what are the faithfulness metrics for a given set of used explanations?

[Relevant metafields](anchor_metafields_evaluate)